# Introduction

This lesson demonstrates how to use the SDK to interact with the Orchestration Service, enabling the creation of AI-driven workflows by seamlessly integrating various modules, such as templating, large language models (LLMs), data masking and content filtering. By leveraging these modules, you can build complex, automated workflows that enhance the capabilities of your AI solutions. For more details on configuring and using these modules, please refer to the [Orchestration Service Documentation](https://help.sap.com/docs/sap-ai-core/sap-ai-core-service-guide/orchestration?locale=en-US).


### ⚠️  Prerequisites
Ensure that the workbook lesson **Orchestration Consumption - Gen AI SDK** steps are followed carefully before continuing this hands-on!

## Setup and configuration

The following Python modules are to be installed during this hands-on introduction. 

#### **generative-ai-hub-sdk**

With this SAP python SDK you can leverage the power of generative Models like chatGPT available in SAP's generative AI Hub.


<br>

> **Note:** Jupyter Notebook kernel restart required after package installation.


</br>

#### Install Python packages

Run the following package installations. **pip** is the package installer for Python. You can use pip to install packages from the Python Package Index and other indexes.

In [ ]:
!pip install generative-ai-hub-sdk --break-system-packages
!pip install ipywidgets --break-system-packages
!pip install pandas --break-system-packages

# kernel restart required!!!


### Verify SDK version

Run the following:

In [ ]:
!pip show generative-ai-hub-sdk

#### Restart Python kernel

The Python kernel needs to be restarted before continuing. 

> ![title](./images/config_001.png)

</br>

> **Note** This will take a couple of minutes.

In [ ]:
# Test embeddings

from gen_ai_hub.proxy.native.openai import embeddings

response = embeddings.create(
    input="SAP Generative AI Hub is awesome!",
    model_name="text-embedding-ada-002"
    
)
print(response.data)

## Initializing the Orchestration Service

Before using the SDK, you need to set up a deployment of the Orchestration Service. 

This is have been done on and you will use a predefined service. Use the pre-deployed access unique endpoint **YOUR_API_URL** in the lesson.

<br>

>**Note:** Find the **YOUR_API_URL** endpoint in the **Orchestration Consumption - Gen AI SDK** lesson **Stap 8**! 

Copy the endpoint and paste it in below:

In [ ]:
YOUR_API_URL = input("Enter YOUR_API_URL from the lesson")
print(YOUR_API_URL)

### Step 1: Define the Template and Default Input Values

The Template class is used to define structured message templates for generating dynamic interactions with language models. In this example, the template is designed for a translation assistant, allowing users to specify a language and text for translation.

In [4]:
from gen_ai_hub.orchestration.models.message import SystemMessage, UserMessage
from gen_ai_hub.orchestration.models.template import Template, TemplateValue

template = Template(
    messages=[
        SystemMessage("You are a helpful translation assistant."),
        UserMessage(
            "Translate the following text to {{?to_lang}}: {{?text}}"
        ),
    ],
    defaults=[
        TemplateValue(name="to_lang", value="German"),
    ],
)

>This template can be used to create translation requests where the language and text to be translated are specified dynamically. The placeholders in the UserMessage will be replaced with the actual values provided at runtime, and the default value for the language is set to German.

### Step 2: Define the LLM

The LLM class is used to configure and initialize a language model for generating text based on specific parameters. In this example, we'll use the GPT-4o model to perform the translation task.

ℹ️Note that virtual deployment of the language model is managed automatically by the Orchestration Service, so no additional deployment setup is required on your part.

In [5]:
from gen_ai_hub.orchestration.models.llm import LLM

llm = LLM(name="gpt-4o", version="latest", parameters={"max_tokens": 256, "temperature": 0.2})

>This configuration initializes the language model to use the gpt-4o variant with the latest updates. The model will generate responses up to 256 tokens in length and produce more predictable and focused output due to the low temperature setting.

### Step 3: Create the Orchestration Configuration

The OrchestrationConfig class is used to create a configuration that integrates various components, such as templates and language models, into a unified orchestration setup. This configuration specifies how these components work together to achieve the desired workflow.

In [6]:
from gen_ai_hub.orchestration.models.config import OrchestrationConfig

config = OrchestrationConfig(
    template=template,
    llm=llm,
)

### Step 4: Run the Orchestration Request

The OrchestrationService class is used to interact with the orchestration service by providing a configuration and invoking its operations. This service handles the execution of workflows defined by the provided configuration and processes inputs accordingly.

In [7]:
from gen_ai_hub.orchestration.service import OrchestrationService

orchestration_service = OrchestrationService(api_url=YOUR_API_URL, config=config)

Call the run method with the required template_values. The service will process the input according to the configuration and return the result.

In [ ]:
result = orchestration_service.run(template_values=[
    TemplateValue(name="text", value="The Orchestration Service is working!")
])
print(result.orchestration_result.choices[0].message.content)

### Data Masking

The Data Masking Module anonymizes or pseudonymizes personally identifiable information (PII) before it is processed by the LLM module. When data is anonymized, all identifying information is replaced with placeholders (e.g., MASKED_ENTITY), and the original data cannot be recovered, ensuring that no trace of the original information is retained. In contrast, pseudonymized data is substituted with unique placeholders (e.g., MASKED_ENTITY_ID), allowing the original information to be restored if needed. In both cases, the masking module identifies sensitive data and replaces it with appropriate placeholders before further processing.

In [9]:
from gen_ai_hub.orchestration.utils import load_text_file
from gen_ai_hub.orchestration.models.data_masking import DataMasking
from gen_ai_hub.orchestration.models.sap_data_privacy_integration import SAPDataPrivacyIntegration, MaskingMethod, \
    ProfileEntity

data_masking = DataMasking(
    providers=[
        SAPDataPrivacyIntegration(
            method=MaskingMethod.ANONYMIZATION,  # or MaskingMethod.PSEUDONYMIZATION
            entities=[
                ProfileEntity.EMAIL,
                ProfileEntity.PHONE,
                ProfileEntity.PERSON,
                ProfileEntity.ORG,
                ProfileEntity.LOCATION
            ]
        )
    ]
)

config = OrchestrationConfig(
    template=Template(
        messages=[
            SystemMessage("You are a helpful AI assistant."),
            UserMessage("Summarize the following CV in 10 sentences: {{?orgCV}}"),
        ]
    ),
    llm=LLM(
        name="gpt-4o",
    ),
    data_masking=data_masking
)

cv_as_string = load_text_file("data/cv.txt")

result = orchestration_service.run(
    config=config,
    template_values=[
        TemplateValue(name="orgCV", value=cv_as_string)
    ]
)

In [ ]:
from IPython.display import Markdown

display(Markdown(result.orchestration_result.choices[0].message.content))

### Content Filtering

The Content Filtering Module can be configured to filter both the input to the LLM module (input filter) and the output generated by the LLM (output filter). The module uses predefined classification services to detect inappropriate or unwanted content, allowing flexible configuration through customizable thresholds. These thresholds can be set to control the sensitivity of filtering, ensuring that content meets desired standards before it is processed or returned as output.

In [ ]:
from gen_ai_hub.orchestration.models.azure_content_filter import AzureContentFilter, AzureThreshold

input_filter= AzureContentFilter(hate=AzureThreshold.ALLOW_SAFE,
                                  violence=AzureThreshold.ALLOW_SAFE,
                                  self_harm=AzureThreshold.ALLOW_SAFE,
                                  sexual=AzureThreshold.ALLOW_SAFE)
output_filter = AzureContentFilter(hate=AzureThreshold.ALLOW_SAFE,
                                   violence=AzureThreshold.ALLOW_SAFE_LOW,
                                   self_harm=AzureThreshold.ALLOW_SAFE_LOW_MEDIUM,
                                   sexual=AzureThreshold.ALLOW_ALL)

config = OrchestrationConfig(
    template=Template(
        messages=[
            SystemMessage("You are a helpful AI assistant."),
            UserMessage("{{?text}}"),
        ]
    ),
    llm=LLM(
        name="gpt-4o",
    ),
    input_filters=[input_filter],
    output_filters=[output_filter]
)

In [ ]:
from gen_ai_hub.orchestration.exceptions import OrchestrationError

try:
    result = orchestration_service.run(config=config, template_values=[
        TemplateValue(name="text", value="I hate you")
    ])
except OrchestrationError as error:
    print(error.message)

## Advanced Examples

### Translation Service

In [19]:
service = OrchestrationService(api_url=YOUR_API_URL)

from gen_ai_hub.orchestration.models.config import OrchestrationConfig
from gen_ai_hub.orchestration.models.llm import LLM
from gen_ai_hub.orchestration.models.message import SystemMessage, UserMessage
from gen_ai_hub.orchestration.models.template import Template, TemplateValue
from gen_ai_hub.orchestration.service import OrchestrationService


class TranslationService:
    def __init__(self, orchestration_service: OrchestrationService):
        self.service = orchestration_service
        self.config = OrchestrationConfig(
            template=Template(
                messages=[
                    SystemMessage("You are a helpful translation assistant."),
                    UserMessage(
                        "Translate the following text to {{?to_lang}}: {{?text}}"
                    ),
                ],
                defaults=[
                    TemplateValue(name="to_lang", value="English"),
                ],
            ),
            llm=LLM(name="gpt-4o"),
        )

    def translate(self, text, to_lang):
        response = self.service.run(
            config=self.config,
            template_values=[
                TemplateValue(name="to_lang", value=to_lang),
                TemplateValue(name="text", value=text),
            ],
        )

        return response.orchestration_result.choices[0].message.content

translator = TranslationService(orchestration_service=service)

In [ ]:
result = translator.translate(text="Hello, world!", to_lang="Afrikaans")
print(result)

In [ ]:
result = translator.translate(text="Hello, world!", to_lang="German")
print(result)

### Chatbot with Memory

In [25]:
from typing import List

from gen_ai_hub.orchestration.models.config import OrchestrationConfig
from gen_ai_hub.orchestration.models.llm import LLM
from gen_ai_hub.orchestration.models.message import Message, SystemMessage, UserMessage
from gen_ai_hub.orchestration.models.template import Template, TemplateValue
from gen_ai_hub.orchestration.service import OrchestrationService


class ChatBot:
    def __init__(self, orchestration_service: OrchestrationService):
        self.service = orchestration_service
        self.config = OrchestrationConfig(
            template=Template(
                messages=[
                    SystemMessage("You are a helpful chatbot assistant."),
                    UserMessage("{{?user_query}}"),
                ],
            ),
            llm=LLM(name="gpt-4o"),
        )
        self.history: List[Message] = []

    def chat(self, user_input):
        response = self.service.run(
            config=self.config,
            template_values=[
                TemplateValue(name="user_query", value=user_input),
            ],
            history=self.history,
        )

        message = response.orchestration_result.choices[0].message

        self.history = response.module_results.templating
        self.history.append(message)

        return message.content
    
bot = ChatBot(orchestration_service=service)

In [ ]:
print(bot.chat("Hello, how are you?"))

In [ ]:
print(bot.chat("What's the weather like today?"))

### Sentiment Analysis with Few Shot Learning 

In [28]:
from typing import List, Tuple

from gen_ai_hub.orchestration.models.config import OrchestrationConfig
from gen_ai_hub.orchestration.models.llm import LLM
from gen_ai_hub.orchestration.models.message import (
    SystemMessage,
    UserMessage,
    AssistantMessage,
)
from gen_ai_hub.orchestration.models.template import Template, TemplateValue
from gen_ai_hub.orchestration.service import OrchestrationService


class FewShotLearner:
    def __init__(
            self,
            orchestration_service: OrchestrationService,
            system_message: SystemMessage,
            examples: List[Tuple[UserMessage, AssistantMessage]],
    ):
        self.service = orchestration_service
        self.config = OrchestrationConfig(
            template=self._create_few_shot_template(system_message, examples),
            llm=LLM(name="gpt-4o"),
        )

    @staticmethod
    def _create_few_shot_template(
            system_message: SystemMessage,
            examples: List[Tuple[UserMessage, AssistantMessage]],
    ) -> Template:
        messages = [system_message]

        for example in examples:
            messages.append(example[0])
            messages.append(example[1])
        messages.append(UserMessage("{{?user_input}}"))

        return Template(messages=messages)

    def predict(self, user_input: str) -> str:
        response = self.service.run(
            config=self.config,
            template_values=[TemplateValue(name="user_input", value=user_input)],
        )

        return response.orchestration_result.choices[0].message.content

In [29]:
sentiment_examples = [
    (UserMessage("I love this product!"), AssistantMessage("Positive")),
    (UserMessage("This is terrible service."), AssistantMessage("Negative")),
    (UserMessage("The weather is okay today."), AssistantMessage("Neutral")),
]

In [30]:
sentiment_analyzer = FewShotLearner(
    orchestration_service=service,
    system_message=SystemMessage(
        "You are a sentiment analysis assistant. Classify the sentiment as Positive, Negative, or Neutral."
    ),
    examples=sentiment_examples,
)

In [ ]:
print(sentiment_analyzer.predict("The movie was a complete waste of time!"))

In [ ]:
print(sentiment_analyzer.predict("I'm not sure how I feel about the recent events."))